# File 02: Fetching Tweets For Each User

In this file we shall be using the 'Twitter API' for fetching tweets from users that we selected in the previous file. This data will further be used to infer the tendency of the user to say something positive or negative.

### Input File:
- 01-select-users.csv -----> contains selected usernames from '1600k-emoticons.csv' dataframe

### Output File:
- 02-user-tweets.csv ------> contains tweets from timeline for usernames dataframe

### Steps:
1. Load Usernames
2. Load API
3. Create Function 'Twitter User' that would
    1. Get UserID from Username
    2. Make Fuction 'get_tweets':
        1. Fetches Tweets from Timeline
        2. Extracts the tweets from the query and save it in an array
        3. Returns the list of tweets
4. Apply 'get_tweets' function on every single 'Username' in the loaded dataframe
5. Save the dataset as '02-user-tweets.csv'

In [3]:
# Loading all Libraries

import pandas as pd
import numpy as np
import tweepy
from tweepy import OAuthHandler
from tqdm import tqdm

In [6]:
# loading dataframe containing all usernames

df = pd.read_csv('db/01-select-users.csv')
df['TWEET'] = None

In [9]:
# setting up Twitter API using 'Tweepy'

consumer_key = 'e3quFb7yTv8RJBfJtcsH172ey'
consumer_secret =  'SI8hYfTDQ6t90DVzk8saJlbp3Frz9eo0IWW9qCBK5JzgLj4ofa'
access_token = '724078891384582144-KG0kZkal2PbRFiXOQva8Uatull9qVRx'
access_token_secret = 'JWZlsm0KYB4vkjzc2CuJOkVaoym0L2Ts2lK9bBhSRMm3t'
try :
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth)
    print("Authentication Successful!")
except :
    print("Authentication Failed!")

Authentication Successful!


In [14]:
# create function to fetch tweets

class Twitter_User():

    def __init__(self,username,count=200):
        self.id = api.get_user(username).id
        self.username = username
        self.count = count
        self.data = None

    def get_tweets(self):
        store_tweets = api.user_timeline(self.id, count=self.count)
        simple_list = []
        for status in store_tweets:
            array = [status._json["text"].strip(), status._json["favorite_count"], status._json["created_at"],status._json["retweet_count"],[h["text"] for h in status._json["entities"]["hashtags"]]]
            simple_list.append(array)
        self.data = pd.DataFrame(simple_list, columns=["Text", "Like", "Created at","Retweet","Hashtags"])
        self.data = self.data[~self.data["Text"].str.startswith('RT')]
        return self.data


    def __repr__(self):
        id = api.get_user(self.id)
        return id.screen_name
    
def fetch_tweets(username) :
    user = Twitter_User(username)
    tweets = user.get_tweets()
    for tweet in tweets['Text']:
        all_tweets.loc[len(all_tweets.index)] = [username, tweet]
        
all_tweets = pd.DataFrame(columns = ['USERNAME', 'TWEETS'])
all_tweets = all_tweets[0:0]

In [ ]:
# Fetching Tweets for Each User

failed = []
for user in tqdm(df['USERNAME']) :
    try :
        fetch_tweets(user)
    except :
        failed.append(user)

In [ ]:
# Analyzing Fetched Data

# print(len(failed))
# print(failed)
# counts = []
# for user in all_tweets['USERNAME']:
#     counts.append(user)
# len(counts)
# my_dict = {i:counts.count(i) for i in counts}
# print(my_dict)
# all_tweets.head()

In [ ]:
# Saving Dataset

all_tweets.to_csv('db/02-user-tweets.csv', index=False)
# print(f"TOTAL NUMBER OF USERS: {len(df) - len(failed)}")